In [17]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix

# Getting the data

In [2]:
messages = pd.read_csv('SMSSpamCollection', sep='\t',
                           names=["label", "message"])

# Preprocesing

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/adelchiasta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

# Converting our sentences into vectors using Bag of words model

In [5]:
# Creating the Bag of Words model
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

In [6]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Encoding label

In [23]:
y=pd.get_dummies(messages['label'])
y=y.iloc[:,1].values # Only one column is enough to consider the whole information

In [24]:
y

array([0, 0, 1, ..., 0, 0, 0], dtype=uint8)

# Train and test split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Model: Naive bayes classifier

In [13]:
spam_detect_model = MultinomialNB().fit(X_train, y_train)

y_pred=spam_detect_model.predict(X_train)


In [14]:
y_pred

array([0, 0, 0, ..., 1, 0, 0], dtype=uint8)

# Accuracy and confusion matrix

In [15]:
print("Train Naive Bayes Accuracy: ", metrics.accuracy_score(y_train, y_pred))

Train Naive Bayes Accuracy:  0.9887816917208885


In [16]:
print("Test Naive Bayes Accuracy: ", metrics.accuracy_score(y_test, spam_detect_model.predict(X_test)))

Test Naive Bayes Accuracy:  0.9838565022421525


In [18]:
confusion_m=confusion_matrix(y_test, spam_detect_model.predict(X_test))

In [19]:
confusion_m

array([[945,  10],
       [  8, 152]])